In [1]:
import pandas as pd 

In [2]:
df=pd.read_csv('Medicine_Details.csv')
df.head()

,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,Poor Review %
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,22
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,18
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,21
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,35
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,29


In [5]:
df_filtered = df[['Medicine Name','Composition' ,'Uses', 'Side_effects']]

In [6]:
df_filtered.head()

,Medicine Name,Composition,Uses,Side_effects
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance


In [ ]:
# df_filtered.to_csv('filtered_medicine_details.csv', index=False)

In [8]:
NEO4J_URI='neo4j+s://5c4048d0.databases.neo4j.io'
NEO4J_USER='neo4j'
NEO4J_PASSWORD='vjetHC2FpoE0-IHaJ-L6z_ShDN1apfA_sFCnNNXOo3E'

In [9]:
import os 
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USER"]=NEO4J_USER
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [13]:
from langchain_neo4j import Neo4jGraph


graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USER, password=NEO4J_PASSWORD)
graph 

In [20]:
medicine_query = """
LOAD CSV WITH HEADERS FROM
'https://drive.google.com/uc?export=download&id=14oHvT-j3Du4fgOKBol6yOtkgFGgChRg7'
AS row

// Medicine node
MERGE (m:Medicine {name: row.`Medicine Name`})

// Uses: comma-separated
FOREACH (use IN split(row.Uses, ',') |
    MERGE (u:Use {name: trim(use)})
    MERGE (m)-[:USED_FOR]->(u)
)

// Composition: split components on '+'
FOREACH (comp IN split(row.Composition, '+') |
    MERGE (c:Composition {name: trim(comp)})
    MERGE (m)-[:CONTAINS]->(c)
)

// Side effects: comma-separated
FOREACH (se IN split(row.Side_effects, ',') |
    MERGE (s:SideEffect {name: trim(se)})
    MERGE (m)-[:HAS_SIDE_EFFECT]->(s)
)
"""


In [21]:
medicine_query

"\nLOAD CSV WITH HEADERS FROM\n'https://drive.google.com/uc?export=download&id=14oHvT-j3Du4fgOKBol6yOtkgFGgChRg7'\nAS row\n\n// Medicine node\nMERGE (m:Medicine {name: row.`Medicine Name`})\n\n// Uses: comma-separated\nFOREACH (use IN split(row.Uses, ',') |\n    MERGE (u:Use {name: trim(use)})\n    MERGE (m)-[:USED_FOR]->(u)\n)\n\n// Composition: split components on '+'\nFOREACH (comp IN split(row.Composition, '+') |\n    MERGE (c:Composition {name: trim(comp)})\n    MERGE (m)-[:CONTAINS]->(c)\n)\n\n// Side effects: comma-separated\nFOREACH (se IN split(row.Side_effects, ',') |\n    MERGE (s:SideEffect {name: trim(se)})\n    MERGE (m)-[:HAS_SIDE_EFFECT]->(s)\n)\n"

In [22]:
graph.query(medicine_query)

[]

In [24]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Medicine {name: STRING}
Use {name: STRING}
Composition {name: STRING}
SideEffect {name: STRING}
Relationship properties:

The relationships:
(:Medicine)-[:USED_FOR]->(:Use)
(:Medicine)-[:CONTAINS]->(:Composition)
(:Medicine)-[:HAS_SIDE_EFFECT]->(:SideEffect)


In [25]:
import os 
from dotenv import load_dotenv
load_dotenv()
PPLX_API_KEY=os.getenv("PPLX_API_KEY")

In [36]:
from langchain_perplexity import ChatPerplexity
llm=ChatPerplexity(api_key=PPLX_API_KEY,model="sonar")
llm

ChatPerplexity(profile={'max_input_tokens': 128000, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False}, client=<openai.OpenAI object at 0x7cc54c9e2c60>, model_kwargs={}, pplx_api_key=SecretStr('**********'))

In [37]:
from langchain_neo4j import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm, graph=graph,verbose=True,
                                  allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x7cc66052f5f0>, cypher_generation_chain=PromptTemplate(input_variables=['examples', 'question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nExamples (optional):\n{examples}\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatPerplexity(profile={'max_input_tokens': 128000, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': Fals

In [38]:
response=chain.run("Which medicine is used for fever?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Medicine)-[:USED_FOR]->(u:Use {name: 'fever'})
RETURN m.name

Full Context:
[]

> Finished chain.


"**Common over-the-counter medicines used for fever include acetaminophen (e.g., Tylenol), ibuprofen (e.g., Advil), naproxen (e.g., Aleve), and aspirin (e.g., Bayer).**[1][2][3][4][7][8]\n\nThese fall into two main categories: acetaminophen, which reduces fever by acting on the brain's pain and temperature centers, and NSAIDs (nonsteroidal anti-inflammatory drugs) like ibuprofen, naproxen, and aspirin, which lower fever by inhibiting prostaglandins that cause inflammation and elevated temperature.[1][2][4]\n\n### Key Options and Considerations\n- **Acetaminophen**: Popular for adults and children, available in tablets or liquid; low stomach irritation risk but follow dosing to avoid liver issues.[1][2][4][7]\n- **Ibuprofen**: NSAID effective for fever, pain, and inflammation; suitable for adults but check for stomach, kidney, or heart conditions.[1][2][3][4][7]\n- **Naproxen**: Longer-lasting NSAID for fever and pain; take as directed.[2][3][4]\n- **Aspirin**: NSAID for fever reduction

In [39]:
response=chain.run("Which medicine is used for malaria?")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Medicine)-[:USED_FOR]->(u:Use {name: 'malaria'})
RETURN m.name

Full Context:
[]

> Finished chain.
**No single medicine is universally used for malaria; treatment depends on the parasite type, resistance patterns, location, and patient factors like age or pregnancy.**[1][2][3]

Common treatments include:
- **Artemisinin-based combination therapies (ACTs)**, such as artemether-lumefantrine (Coartem), often the first-line for chloroquine-resistant *P. falciparum*.[1][2][3]
- **Chloroquine phosphate**, preferred for sensitive parasites (e.g., some *P. vivax*, *P. malariae*), but ineffective where resistance is common.[1][2][3][7]
- **Atovaquone-proguanil (Malarone)**, a common option for uncomplicated cases.[1][2][3][6]
- **Quinine sulfate** combined with doxycycline, tetracycline, or clindamycin, used as a second-line for resistant *P. falciparum*.[1][2][3]

Other options like primaquine (for liver stages), m

In [41]:
response=chain.run("what is composition of medicine used for diabetes?")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Medicine)-[:USED_FOR]->(u:Use {name: 'diabetes'})-[:CONTAINS]->(c:Composition)
RETURN m.name, c.name

Full Context:
[]

> Finished chain.
**Medicines for diabetes primarily include several classes for managing type 1 and type 2 diabetes, such as biguanides like metformin, sulfonylureas, SGLT2 inhibitors, GLP-1 receptor agonists, insulins, and various combinations.[1][2][3]**

### Main Drug Classes and Examples
Diabetes medications work by different mechanisms, like lowering glucose production in the liver, increasing insulin sensitivity, stimulating insulin release, or promoting glucose excretion. Here's a breakdown of key classes:

- **Biguanides**: Metformin (Glucophage) is the most common; it reduces liver glucose production and improves insulin sensitivity in muscles.[2][6][7]
- **Sulfonylureas**: Stimulate the pancreas to release more insulin. Examples: glimepiride (Amaryl), glipizide (Glucotrol, Glucot